In [7]:
import torch

from lib_all import *

## Xây dựng hàm decode

In [8]:
def decode(default_boxes, offsets):
    """
    :param default_boxes:[8732, 4]
    :param offsets: [8732, 4]  trong do 4 la delta cx, delta cy, delta h, delta w
    :return: [8732, 4] cac thong so bbox
    """
    #tinh cx, cy, h, w
    boxes = torch.cat(
        default_boxes[:,:2]( 1 +  0.1 * offsets[:,:2]),
        default_boxes[:,2:]*torch.exp(0.2*offsets[:,2:]),
        dim = 1 #cat theo hang ngang
    )
    # chuyen ve dang x_min, y_min, x_max, y_max
    boxes [:, :2]  -= boxes[:, 2:] # x_min, y_ min
    boxes[:, 2:] +=boxes[:,:2]
    return boxes

## Xây dựng hàm non-maximum-superestion

Loại đi những cái không phải độ tự tin cao nhất

In [9]:
def nms(boxes, confident, threshold = 0.45, top_k = 200):
    """
    :param boxes: [8732, 4] cac thong so bbox
    :param confident: [8732] do tu tin cua tung bbox
    :param threshold: nguong overlap
    :param top_k:
    :return: tensor
    """
    count = 0
    keep = confident.new(confident.size()).zero_().long() # tensor_type giong confident, so chieu confident.size(), toan bo la so 0, kieu long

    # thong so box
    x_min = boxes[:, 0]
    y_min = boxes[:, 1]
    x_max = boxes[:, 2]
    y_max = boxes[:, 3]
    area = torch.mul((x_max-x_min),(y_max-y_min))

    tmp_x_min = boxes.new()
    tmp_y_min = boxes.new()
    tmp_x_max = boxes.new()
    tmp_x_max = boxes.new()
    tmp_w = boxes.new()
    tmp_h = boxes.new()

    value, index = confident.sort(0)
    index = index[-top_k:] # lay ra 200 cai cuoi co do tu tin cao nhat

    while index.numel() > 0:
        #Luu bien lon nhat
        keep[count] = index[-1]
        count +=1

        if index.size(0) == 1: # neu chi con 1 phan tu thi khong con gi de ma so sanh
            break

        index = inden[:-1] # loai bo box co confident cao nhat